In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import random_split, DataLoader
import time
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [16]:
# Define a CNN model with configurable activation functions for each layer
class CNNModel(nn.Module):
    def __init__(self, activations=None):
        super(CNNModel, self).__init__()

        if activations is None:
            activations = [nn.ReLU(), nn.ReLU(), nn.ReLU(), nn.ReLU()]

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1, stride=1),
            activations[0],
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1, stride=1),
            activations[1],
            nn.MaxPool2d(2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=1, padding=1),
            activations[2],
            nn.MaxPool2d(2)
        )

        self.dense1 = nn.Sequential(
            nn.Linear(128, 128),
            activations[3]
        )
        self.dense2 = nn.Sequential(
            nn.Linear(128, 10),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.dense1(out)
        out = self.dense2(out)
        return out

In [17]:
model = CNNModel().to(device)

In [6]:
def initialize_weights_uniform(model):
    a = -0.1
    b = 0.1
    for module in model.modules():
        # Initialize convolutional layers
        if isinstance(module, nn.Conv2d):
            nn.init.uniform_(module.weight, a=a, b=b)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        
        # Initialize linear (dense) layers
        elif isinstance(module, nn.Linear):
            nn.init.uniform_(module.weight, a=a, b=b)
            if module.bias is not None:
                nn.init.zeros_(module.bias)